In [8]:
import os
import sys
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis\read")
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis")

import numpy as np
import matplotlib.pyplot as plt
import scipy

from read_scope_data import read_trc_data
from data_analysis_utils import Photons, calculate_stft
from plot_utils import plot_counts_per_bin, plot_photon_detection

%matplotlib qt

In [ ]:
# Define parameters for file selection
date = "20241102"
file_number = "00012"
position = "p30-z13-x200"  # Change this for different positions

# Define base pattern for file names
xray_pattern = f"C{{channel}}--E-ring-{position}-xray--{file_number}.trc"
bdot_pattern = f"C{{channel}}--E-ring-{position}-Bdot--{file_number}.trc"

base_dir = r"E:\good_data\He3kA_B250G500G_pl0t20_uw15t35_P30"
print(f"Looking for files with {position}, shot: {file_number}")

# read data from xray detector scope

for channel in ["2", "3"]:  # C2 and C3 for x-ray
    filename = xray_pattern.format(channel=channel)
    filepath = os.path.join(base_dir, filename)
    if os.path.exists(filepath):
        print(f"Reading x-ray data from {filename}")
        if channel == "2":
            dipole_data, tarr_x = read_trc_data(filepath)
        elif channel == "3":
            xray_data, tarr_x = read_trc_data(filepath)
    else:
        print(f"Warning: Could not find {filename}")


for channel in ["1", "2", "3"]:  # C1, C2, and C3 for Bdot
    filename = bdot_pattern.format(channel=channel)
    filepath = os.path.join(base_dir, filename)
    if os.path.exists(filepath):
        print(f"Reading Bdot data from {filename}")
        if channel == "1":
            By_P21, tarr_B = read_trc_data(filepath)
        elif channel == "2":
            Bx_P20, tarr_B = read_trc_data(filepath)
        elif channel == "3":
            By_P20, tarr_B = read_trc_data(filepath)
    else:
        print(f"Warning: Could not find {filename}")

print("\nData loading complete")

In [ ]:
# Create Photons detector object
detector = Photons(tarr_x * 1000, -xray_data, threshold_multiplier=5.0, negative_pulses=False)

# Reduce pulses to get pulse properties
detector.reduce_pulses()

# Get pulse arrays
pulse_times, pulse_areas = detector.get_pulse_arrays()

plot_photon_detection(
    time_array=tarr_x,
    signal_data=xray_data,
    pulse_times=pulse_times,
    threshold_level=detector.threshold,
    offset=detector.offset
)

In [ ]:
# Calculate STFT for each Bdot signal
freq_bins = 500
overlap_fraction = 0.05
freq_min = 100e6  # 100 MHz
freq_max = 800e6  # 800 MHz

# Calculate STFT for By_P21
freq1, stft_matrix1, stft_time1 = calculate_stft(
    time_array=tarr_B,
    data_arr=By_P21,
    freq_bins=freq_bins,
    overlap_fraction=overlap_fraction,
    window='hanning',
    freq_min=freq_min,
    freq_max=freq_max
)

# Calculate STFT for Bx_P20
freq2, stft_matrix2, stft_time2 = calculate_stft(
    time_array=tarr_B,
    data_arr=Bx_P20,
    freq_bins=freq_bins,
    overlap_fraction=overlap_fraction,
    window='hanning',
    freq_min=freq_min,
    freq_max=freq_max
)

# Calculate STFT for By_P20
freq3, stft_matrix3, stft_time3 = calculate_stft(
    time_array=tarr_B,
    data_arr=By_P20,
    freq_bins=freq_bins,
    overlap_fraction=overlap_fraction,
    window='hanning',
    freq_min=freq_min,
    freq_max=freq_max
)

# Create a figure with 3 subplots
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15))

# Plot STFT for By_P21
im1 = ax1.imshow(stft_matrix1.T,
                 aspect='auto',
                 origin='lower',
                 extent=[stft_time1[0]*1e3, stft_time1[-1]*1e3, freq1[0]/1e6, freq1[-1]/1e6],
                 interpolation='None',
                 cmap='jet')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Frequency (MHz)')
ax1.set_title('STFT of By_P21')
fig.colorbar(im1, ax=ax1, label='Magnitude')

# Plot STFT for Bx_P20
im2 = ax2.imshow(stft_matrix2.T,
                 aspect='auto',
                 origin='lower',
                 extent=[stft_time2[0]*1e3, stft_time2[-1]*1e3, freq2[0]/1e6, freq2[-1]/1e6],
                 interpolation='None',
                 cmap='jet')
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel('Frequency (MHz)')
ax2.set_title('STFT of Bx_P20')
fig.colorbar(im2, ax=ax2, label='Magnitude')

# Plot STFT for By_P20
im3 = ax3.imshow(stft_matrix3.T,
                 aspect='auto',
                 origin='lower',
                 extent=[stft_time3[0]*1e3, stft_time3[-1]*1e3, freq3[0]/1e6, freq3[-1]/1e6],
                 interpolation='None',
                 cmap='jet')
ax3.set_xlabel('Time (ms)')
ax3.set_ylabel('Frequency (MHz)')
ax3.set_title('STFT of By_P20')
fig.colorbar(im3, ax=ax3, label='Magnitude')

# Adjust layout
plt.tight_layout()
plt.show()